In [0]:
spark_df = spark.read.table("hive_metastore.default.az_test_df_csv")
pandas_df = spark_df.toPandas()

In [0]:
pandas_df

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Water_Cement,Coarse_Fine,Aggregate,Aggregate_Cement,Slag_Cement,Ash_Cement,Plastic_Cement,Age_Water
0,166.1,75.4,163.8,173.8,4.6,1007.2,746.6,56,1.046358,1.349049,1753.8,10.558700,0.453943,0.986153,0.027694,0.322209
1,304.0,0.0,0.0,190.0,0.0,998.0,801.0,7,0.625000,1.245943,1799.0,5.917763,0.000000,0.000000,0.000000,0.036842
2,225.0,0.0,0.0,185.0,0.0,1113.0,833.0,28,0.822222,1.336134,1946.0,8.648889,0.000000,0.000000,0.000000,0.151351
3,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,0.749801,1.357265,1786.1,7.104614,0.000000,0.470565,0.025457,0.530504
4,144.0,15.0,195.0,176.0,6.0,1021.0,709.0,28,1.222222,1.440056,1730.0,12.013889,0.104167,1.354167,0.041667,0.159091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600,140.0,129.0,100.0,200.0,7.0,864.0,753.0,28,1.428571,1.147410,1617.0,11.550000,0.921429,0.714286,0.050000,0.140000
3601,281.0,0.0,0.0,186.0,0.0,1104.0,774.0,28,0.661922,1.426357,1878.0,6.683274,0.000000,0.000000,0.000000,0.150538
3602,289.0,133.0,0.0,194.0,7.0,924.0,760.0,28,0.671280,1.215789,1684.0,5.826990,0.460208,0.000000,0.024221,0.144330
3603,469.0,117.2,0.0,137.8,32.2,852.1,840.5,3,0.293817,1.013801,1692.6,3.608955,0.249893,0.000000,0.068657,0.021771


In [0]:
import mlflow
logged_model_lgbm = 'runs:/17f73d1069a94712ab86249773b1588c/model'
logged_model_xgb = 'runs:/a16b6dcfd350484d9969d3a623ac64f4/model'
logged_model_forest_regressor = 'runs:/af59677710734ac4a2d4b20cfb42c784/model'

logged_model_lgbm = mlflow.pyfunc.load_model(logged_model_lgbm)
logged_model_xgb = mlflow.pyfunc.load_model(logged_model_xgb)
logged_model_forest_regressor = mlflow.pyfunc.load_model(logged_model_forest_regressor)



In [0]:
from sklearn.ensemble import VotingRegressor
voting_regressor = VotingRegressor(
    estimators=[('model1', logged_model_lgbm), ('model2', logged_model_xgb), ('model3', logged_model_forest_regressor)]
)

In [0]:
y_pred_lgbm = logged_model_lgbm.predict(pandas_df)
y_pred_xgb = logged_model_xgb.predict(pandas_df)
y_pred_forest_regressor = logged_model_forest_regressor.predict(pandas_df)

In [0]:
y_pred = (1/3 * y_pred_lgbm) + (1/3 * y_pred_xgb) + (1/3 * y_pred_forest_regressor)

In [0]:
y_pred

Out[11]: array([46.68788053, 20.1642651 , 31.77126036, ..., 40.64685409,
       31.26666067, 21.90627009])

In [0]:
import numpy as np
import pandas as pd
ids = np.arange(5407, 5407 + len(y_pred))

submission = pd.concat([pd.DataFrame(ids, columns=['id']), pd.DataFrame(y_pred, columns=['Strength'])], axis=1)

In [0]:
submission

,id,Strength
0,5407,46.687881
1,5408,20.164265
2,5409,31.771260
3,5410,48.300204
4,5411,31.463126
...,...,...
3600,9007,31.461344
3601,9008,34.191077
3602,9009,40.646854
3603,9010,31.266661


In [0]:
submission.to_csv('submission.csv',index=False)

In [0]:
submission

,id,Strength
0,5407,46.687881
1,5408,20.164265
2,5409,31.771260
3,5410,48.300204
4,5411,31.463126
...,...,...
3600,9007,31.461344
3601,9008,34.191077
3602,9009,40.646854
3603,9010,31.266661


In [0]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()

# Convert the Pandas DataFrame to a Spark DataFrame
df_spark = spark.createDataFrame(submission)

# Preview the Spark DataFrame
df_spark.show()

+----+------------------+
|  id|          Strength|
+----+------------------+
|5407| 46.68788053350171|
|5408| 20.16426509693051|
|5409|31.771260358383905|
|5410| 48.30020405450931|
|5411|31.463125882153285|
|5412| 42.11395457828815|
|5413| 36.64731813085305|
|5414|23.161972508961146|
|5415| 48.46788309207019|
|5416| 38.66781525023354|
|5417| 40.62616068252852|
|5418| 58.69460386201854|
|5419| 41.33254324923894|
|5420| 44.93118676165925|
|5421| 30.78545720961143|
|5422|27.608242098055747|
|5423| 49.92307998049682|
|5424|45.487834000627764|
|5425| 37.15673568314517|
|5426| 49.39793939322557|
+----+------------------+
only showing top 20 rows



In [0]:
print(spark_df)

DataFrame[CementComponent: double, BlastFurnaceSlag: double, FlyAshComponent: double, WaterComponent: double, SuperplasticizerComponent: double, CoarseAggregateComponent: double, FineAggregateComponent: double, AgeInDays: int, Water_Cement: double, Coarse_Fine: double, Aggregate: double, Aggregate_Cement: double, Slag_Cement: double, Ash_Cement: double, Plastic_Cement: double, Age_Water: double]


In [0]:



spark = SparkSession.builder \
    .appName("Hive Metastore Example") \
    .enableHiveSupport() \
    .getOrCreate()

# Connect to the Hive Metastore
spark.sql("use hive_metastore.default")

# Query a table in the database
df = spark.sql("SELECT * FROM filestore")

# Display the data in the table
display(df)

DataFrame[id: int, Strength: double]

In [0]:
df.show()

+----+------------------+
|  id|          Strength|
+----+------------------+
|5407| 46.68788053350171|
|5408| 20.16426509693051|
|5409|31.771260358383905|
|5410| 48.30020405450931|
|5411|31.463125882153285|
|5412| 42.11395457828815|
|5413| 36.64731813085305|
|5414|23.161972508961146|
|5415| 48.46788309207019|
|5416| 38.66781525023354|
|5417| 40.62616068252852|
|5418| 58.69460386201854|
|5419| 41.33254324923894|
|5420| 44.93118676165925|
|5421| 30.78545720961143|
|5422|27.608242098055747|
|5423| 49.92307998049682|
|5424|45.487834000627764|
|5425| 37.15673568314517|
|5426| 49.39793939322557|
+----+------------------+
only showing top 20 rows

